# Training

This notebook demonstrates the process of training an ensemble learning model using a provided CSV file. It showcases data preprocessing, model training, evaluation, and saving the trained model. The ensemble method (hard voting, soft voting, or stacking) can be selected based on the user's choice.


## Importing Necessary Libraries

First, we import all the necessary libraries and modules needed for this script. This includes libraries for handling warnings, data manipulation, machine learning, and the custom Ensemble module containing ensemble learning methods.

In [1]:
import pandas as pd
import torch
import numpy as np
from src import Bayes, Utils

/tmp/ipykernel_26081/2625755908.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Utility Functions

In [2]:
MODEL_FOLDER = 'model_bayes'

### Function to Read CSV File

The `read_csv_file` function reads the CSV file and returns a pandas DataFrame. If the file is not found, the script will exit with an error message.

In [3]:
def read_csv_file(filename: str) -> pd.DataFrame:
    try:
        data = pd.read_csv(filename, lineterminator='\n', usecols=range(2))
        print("CSV file read successfully!")
        return data
    except FileNotFoundError:
        print("ERROR: File not found")
        exit(1)

dataset = Utils.read_csv_file('datasets/datasetall.csv')
dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


In [4]:
dataset['label'].value_counts(ascending=True)

label
0    14115
1    14346
Name: count, dtype: int64

### Function to Seed Random Number Generators

To ensure reproducibility, the `seed_random_number_generators` function seeds the random number generators for PyTorch and NumPy.

In [5]:
def seed_random_number_generators(seed=0):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    print("Random number generators seeded.")

Utils.seed_random_number_generators()

Random number generators seeded.


### Function for Train-Test Split

The `get_train_test_split` function splits the dataset into training and testing sets with an 80-20 split ratio and returns them.

In [6]:
TEST_SIZE = 0.2

def shuffle_data_frame(data_frame):
    text = list(data_frame['text'])
    label = list(data_frame['label'])

    assert(len(text) == len(label))

    indices = list(range(len(label)))

    # Make a random number generator that will shuffle list of indices
    # It is seeded to be reproducible
    random_number_generator = np.random.default_rng(seed=0)
    random_number_generator.shuffle(indices)

    shuffled_text = []
    shuffled_labels = []

    # Iterate through the list of indices and add the original data
    # from those shuffled indices
    for index in indices:
        shuffled_text.append(text[index])
        shuffled_labels.append(label[index])

    return pd.DataFrame({
        'text': shuffled_text,
        'label': shuffled_labels,
    })


def get_train_test_split(data_frame: pd.DataFrame, test_size: float):
    """
    Makes a stratified train test split.
    This aims to preserve the distribution between classes.
    """
    if not (1 > test_size > 0):
        print('ERROR: test_size must be between 0 and 1')
        return

    data_frame = shuffle_data_frame(data_frame)

    data_frame_length = len(data_frame)
    train_size = 1 - test_size

    nonhate_rows = data_frame[data_frame['label'] == 0] 
    nonhate_row_length = len(nonhate_rows)

    nonhate_row_train_size = math.ceil(nonhate_row_length * train_size)

    nonhate_row_train = nonhate_rows[0:nonhate_row_train_size]
    nonhate_row_test = nonhate_rows[nonhate_row_train_size:nonhate_row_length]

    assert(len(nonhate_row_train) + len(nonhate_row_test) == nonhate_row_length)

    hate_rows = data_frame[data_frame['label'] == 1] 
    hate_row_length = len(hate_rows)

    hate_row_train_size = math.ceil(hate_row_length * train_size)

    hate_row_train = hate_rows[0:hate_row_train_size]
    hate_row_test = hate_rows[hate_row_train_size:hate_row_length]

    assert(len(hate_row_train) + len(hate_row_test) == hate_row_length)

    combined_train = pd.concat([nonhate_row_train, hate_row_train])
    combined_test = pd.concat([nonhate_row_test, hate_row_test])

    shuffled_train = shuffle_data_frame(combined_train)
    shuffled_test = shuffle_data_frame(combined_test)

    return (
        shuffled_train['text'],
        shuffled_test['text'],
        shuffled_train['label'],
        shuffled_test['label'],
    )

X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

## Train Data

In [7]:
pd.DataFrame(X_train)

,text
0,[USERNAME] Palangga ka man sang mga taga Baco...
1,Who dafuq is Jose Montemayor Jr.???
2,Di na nakakatuwa yung mukha ni Mar Roxas sa TV...
3,national elections. | via[USERNAME]
4,"Binay will be staring in a movie called ""The D..."
...,...
22764,"""Kala ko wala andito pala si Marcos.""*pertaini..."
22765,sie ~ [USERNAME]Marcos Magnanakaw Marcos Dikta...
22766,If Mar is BatMarBinay is Bane-ay.
22767,to my moots im sorry in not sorry for flooding...


In [8]:
y_train_dataframe = pd.DataFrame(y_train, columns=['label'])
y_train_dataframe

,label
0,0
1,0
2,1
3,0
4,1
...,...
22764,0
22765,1
22766,1
22767,1


In [9]:
y_train_dataframe.value_counts(ascending=True)

label
0        11292
1        11477
Name: count, dtype: int64

## Test Data

In [10]:
VAL_SPLIT = 0.5

X_val, X_test, y_val, y_test = Utils.get_train_test_split(
  pd.DataFrame({
    'text': X_test,
    'label': y_test,
  }), 
  VAL_SPLIT,
)

In [11]:
pd.DataFrame(X_val)

,text
0,let leni lead Jessica Soho nterviews
1,Oy Let Leni Lead daw sabi ni tomi
2,Dapat Si Leni
3,[USERNAME]and[USERNAME] Beautiful girlWise too...
4,[USERNAME] TO DONATE GCash Primitiva C TRANSPA...
...,...
2842,bobong marcos
2843,My kakampink rice
2844,Poor Binay. Too desperate. Hekhek
2845,Yup correlated to IQ Marcos Magnanakaw


In [12]:
y_val_dataframe = pd.DataFrame(y_val, columns=['label'])
y_val_dataframe

,label
0,0
1,0
2,0
3,0
4,0
...,...
2842,1
2843,0
2844,1
2845,1


In [13]:
pd.DataFrame(X_test)

,text
0,Chika natin mga nagawa ni[USERNAME]ha pati sa ...
1,glad that my man is kakampink Leni Kiko
2,Ayan daw mga nagawatalo pa DPWHPakivalidate mg...
3,Rizalito david is for morality ahahahaha
4,Its our choice and our right whom to vote for ...
...,...
2840,Ayaw na nilang lagyan ng mukha ni Binay yung T...
2841,Im one of the Thank you VP[USERNAME]for the in...
2842,[USERNAME] lugaw ka putang ina mo pag nakaupo ...
2843,you know whats a bad joke? mar's shady mrt dea...


In [14]:
y_test_dataframe = pd.DataFrame(y_test, columns=['label'])
y_test_dataframe

,label
0,0
1,0
2,0
3,0
4,0
...,...
2840,1
2841,0
2842,1
2843,1


In [15]:
y_test_dataframe.value_counts(ascending=True)

label
0        1411
1        1434
Name: count, dtype: int64

### Bernoulli Naive Bayes

In [16]:
train_bayes = Bayes.BayesPipeline

train_bayes

Pipeline(steps=[('tfidf', CountVectorizer()),
                ('bayes', BernoulliNB(alpha=0.3))])

In [17]:
train_bayes.fit(X_train, y_train)

Pipeline(steps=[('tfidf', CountVectorizer()),
                ('bayes', BernoulliNB(alpha=0.3))])

In [18]:
accuracy, recall, precision, f1 = Utils.get_prediction_results(
  X_val,
  y_val,
  train_bayes,
)

Accuracy: 0.8236740428521251
Recall: 0.8780487804878049
Precision: 0.7939508506616257
F1-score: 0.8338848444738585


In [19]:
test_accuracy, test_recall, test_precision, test_f1 = Utils.get_prediction_results(
  X_test,
  y_test,
  train_bayes,
)

Accuracy: 0.820738137082601
Recall: 0.8737796373779637
Precision: 0.7920353982300885
F1-score: 0.8309018567639257


In [20]:
Utils.save_trained_model(train_bayes, f"{MODEL_FOLDER}/Bayes")

Ensemble model saved to Pipeline(steps=[('tfidf', CountVectorizer()),
                ('bayes', BernoulliNB(alpha=0.3))]).pkl
